<a href="https://colab.research.google.com/github/Arteechess/Machine-learning-methods/blob/main/221_365_chistyakov_ad_pr_2_introduction_to_the_recurrent_neural_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Практическая работа №2.
Знакомство с рекуррентной нейронной сетью

ФИО: Чистяков Артем Дмитриевич

Группа: 221-365

In [ ]:
# Проверяем версию TensorFlow и убеждаемся, что используем TensorFlow 2.x
%tensorflow_version 2.x

import tensorflow as tf

# Проверяем, доступен ли GPU и выводим информацию о нем
device_name = tf.test.gpu_device_name()

if device_name == '':
    print('No GPU found. Running on CPU.')
else:
    print('Found GPU at: {}'.format(device_name))

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.
Found GPU at: /device:GPU:0


In [ ]:
# Импортируем необходимые библиотеки
from tensorflow.keras.datasets import imdb  # Используйте tensorflow.keras
import numpy as np

# Задаем параметры набора данных
vocabulary_size = 5000  # Ограничиваем размер словаря наиболее часто встречающимися словами
index_from = 3  # Сдвиг индексов, чтобы зарезервировать индексы для специальных токенов (PADDING, START, UNKNOWN)
seed_value = 4 * 22 - 1  # Вычисляем значение seed, как указано в задании (87)

# Загружаем набор данных IMDb
(X_train, y_train), (X_test, y_test) = imdb.load_data(
    path="imdb.npz",  # Путь для сохранения/загрузки архива
    num_words=vocabulary_size,  # Ограничиваем словарь
    skip_top=0,  # Не пропускаем наиболее часто встречающиеся слова
    maxlen=None,  # Не ограничиваем длину отзывов
    seed=seed_value,  # Задаем seed для воспроизводимости результатов
    start_char=1,  # Символ начала последовательности (отзыва)
    oov_char=2,  # Символ для слов, отсутствующих в словаре
    index_from=index_from,  # Сдвиг индексов
)

# Выводим размерности полученных массивов
print("Размерность обучающих данных:")
print('Shape of X train:', X_train.shape)  # Shape of X train: (25000,)
print('Shape of y train:', y_train.shape)  # Shape of y train: (25000,)

print("\nРазмерность тестовых данных:")
print('Shape of X test:', X_test.shape)  # Shape of X test: (25000,)
print('Shape of y test:', y_test.shape)  # Shape of y test: (25000,)

# Дополнительная информация:
print("\nПример отзыва (X_train[0]):")
print(X_train[0])  # Выводит список индексов слов

print("\nМетка для примера отзыва (y_train[0]):")
print(y_train[0])  # Выводит 0 (отрицательный) или 1 (положительный)


17464789/17464789 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Размерность обучающих данных:
Shape of X train: (25000,)
Shape of y train: (25000,)

Размерность тестовых данных:
Shape of X test: (25000,)
Shape of y test: (25000,)

Пример отзыва (X_train[0]):
[1, 134, 504, 2, 4, 1399, 3836, 9, 669, 2032, 18, 112, 2305, 18, 2, 3166, 5, 1576, 2, 803, 60, 53, 2, 9, 15, 3687, 309, 286, 60, 2305, 18, 118, 167, 2964, 8, 135, 4, 22, 1188, 1816, 53, 2, 74, 36, 71, 2, 10, 10, 4, 65, 3277, 4, 2, 5, 2, 7, 2, 2820, 2, 3750, 35, 2045, 298, 255, 2, 33, 86, 34, 41, 2, 336, 5, 95, 34, 41, 4595, 658, 4, 22, 2, 450, 153, 5, 2679, 1424, 23, 2, 1519, 19, 4, 372, 187, 41, 45, 579, 39, 6, 2, 668, 1971, 21, 25, 2, 1092, 15, 45, 6, 2, 2, 2, 10, 10, 4, 65, 9, 35, 221, 31, 2, 19, 1285, 33, 211, 261, 4, 1375, 1730, 3047, 26, 2, 49, 203, 24, 1144, 4, 653, 11, 1163, 949, 4, 598, 130, 21, 13, 161, 330, 60, 151, 729, 1500, 11, 6, 2911, 22, 62, 1329, 28, 72, 2660, 61, 523, 10, 10, 4, 22, 1990, 289, 735, 2, 18, 116, 2, 3750, 118, 5

In [ ]:
from tensorflow.keras.datasets import imdb

# Загружаем набор данных IMDb (с теми же параметрами, что и ранее)
vocabulary_size = 5000
index_from = 3
seed_value = 4 * 22 - 1

(X_train, y_train), (X_test, y_test) = imdb.load_data(
    num_words=vocabulary_size,
    seed=seed_value,
    index_from=index_from
)

# Выбираем индекс отзыва для анализа
review_index = 90

# 1. Выводим один отзыв из обучающего множества в виде списка индексов слов
print(f"Отзыв {review_index} в виде списка индексов слов:")
print(X_train[review_index])

# 2. Преобразуем список индексов в текст и выводим отзыв
# Получаем словарь соответствия индекса слову
word_to_index = imdb.get_word_index()

# Сдвигаем индексы, чтобы учесть сдвиг index_from
index_to_word = {index + index_from: word for (word, index) in word_to_index.items()}

# Добавляем специальные токены в словарь
index_to_word[0] = "#PAD"  # Padding (заполнитель)
index_to_word[1] = "#START"  # Start (начало отзыва)
index_to_word[2] = "#OOV"  # Out-of-vocabulary (неизвестное слово)
index_to_word[3] = "#UNK"  # Unknown (не используется в данном примере, но обычно для неизвестных слов)

# Преобразуем индексы в слова
indexed_words = [index_to_word[i] for i in X_train[review_index]]

# Соединяем слова в текст
text_review = ' '.join(indexed_words)
print(f"\nОтзыв {review_index} в виде текста:")
print(text_review)

# 3. Выводим длину отзыва
print(f"\nДлина отзыва {review_index}: {len(X_train[review_index])} слов")

# 4. Выводим метку класса данного отзыва и название класса
review_label = y_train[review_index]

# Определяем название класса на основе метки
class_name = "Положительный" if review_label == 1 else "Отрицательный"

print(f"\nМетка класса отзыва {review_index}: {review_label} ({class_name})")


Отзыв 90 в виде списка индексов слов:
[1, 225, 38, 111, 1566, 857, 44, 789, 245, 914, 11, 133, 13, 43, 191, 789, 1858, 101, 1207, 5, 387, 14, 2, 593, 1977, 6, 499, 25, 774, 557, 246, 10, 10, 3479, 2, 102, 26, 61, 2523, 1742, 18, 6, 606, 282, 36, 26, 404, 108, 18, 6, 1050, 2, 88, 13, 92, 28, 8, 104, 54, 13, 106, 98, 15, 817, 13, 92, 28, 8, 3250, 44, 1012, 6, 1315, 114, 88, 50, 115, 9, 6, 2261, 114, 8, 380, 19, 13, 70, 43, 1968, 125, 61, 3730, 4003, 6, 3640, 870, 145, 4416, 125, 61, 4081, 5, 358, 32, 4, 831, 2, 5, 4622, 776, 170, 741, 2, 1686, 3729, 39, 4, 2, 2068, 7, 98, 32, 10, 10, 11, 61, 1151, 789, 245, 914, 407, 734, 83, 4, 38, 78, 45, 52, 2385, 150, 48, 335, 40, 72, 5, 1144, 4, 1157, 1107, 14, 9, 4, 2095, 2, 1250, 585, 45, 38, 78, 15, 45, 220, 2, 8, 106, 5, 43, 54, 25, 104, 15, 12, 191, 869, 79, 101, 433, 12, 2459, 25, 11, 4, 91, 2, 771, 11, 4, 130, 335, 38, 2664, 34, 32, 4, 162, 2191, 7, 3002, 25, 43, 92, 124, 726, 8, 462, 42, 1415, 11, 6, 2, 13, 119, 12, 88, 45, 38, 1543, 1139, 1